In [ ]:
!pip install pandas
!pip install numpy==1.23

In [ ]:
!pip install tensorflow==2.12
!pip install pyarabic

In [ ]:
!pip install transformers

In [ ]:
!pip install ipywidgets
!pip install datasets
!pip install transformers[torch]
!pip install nvidia-ml-py3

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)

In [3]:
# cell-1  
#load and clean the data (removing diacritics and unwanted text)

df = pd.read_csv('poemsDataset.csv')
df.fillna('', inplace=True)
display(len(df))


def remove_diacritics(a):    
    return araby.strip_diacritics(a)

df['first_hemistich'] = df['first_hemistich'].apply(remove_diacritics)
df['second_hemistich'] = df['second_hemistich'].apply(remove_diacritics)

def normalizeBeforeTraining(df):
    df['first_hemistich'] = df['first_hemistich'].str.replace('النابغـة: ', '')
    df['second_hemistich'] = df['second_hemistich'].str.replace('الـربيع: ', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('عبيــد: ', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('امـرؤ القيسـ: ', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('امرؤ القيس: ', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(جلال الــــدين الــــرومي):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(لـوك الفيلسـوف الإنكليزي):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(كانت الفيلسوف الألماني ):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(بركســــــــــــــــون):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(الحـــــــــــــــور):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(الشــــــــــــــاعر):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(الإنســـــــــــــــان):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('العلم):', '', regex=False)
    df['first_hemistich'] = df['first_hemistich'].str.replace('(العشــــــــــــــــق):', '', regex=False)
    df['first_hemistich'] = df['first_hemistich'].str.replace('(الزهــــــــــــــــــرة):', '', regex=False)
    df['second_hemistich'] = df['second_hemistich'].str.replace('التوأم اليشكري: ', '', regex=False)  
    df['first_hemistich'] = df['first_hemistich'].str.replace('آ', 'أ')
    df['second_hemistich'] = df['second_hemistich'].str.replace('آ', 'أ')
    df['first_hemistich'] = df['first_hemistich'].str.replace('[/":?،؟]', '')
    df['second_hemistich'] = df['second_hemistich'].str.replace('[/":?،؟]', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('  ', ' ')
    df['second_hemistich'] = df['second_hemistich'].str.replace('  ', ' ')
    df['first_hemistich'] = df['first_hemistich'].str.replace('  ', ' ')
    df['second_hemistich'] = df['second_hemistich'].str.replace('  ', ' ')


normalizeBeforeTraining(df)
df.drop(df[(df['first_hemistich'] == '') & (df['second_hemistich'] == '')].index, inplace=True)

#if first_hemistich == '', then copy the text from second_hemistich. then delete the text in the second_hemistich
df['first_hemistich'] = df.apply(lambda x: x['second_hemistich'] if x['first_hemistich'] == '' else x['first_hemistich'], axis=1)
df['second_hemistich'] = df.apply(lambda x: '' if x['first_hemistich'] == x['second_hemistich'] else x['second_hemistich'], axis=1)

df.reset_index(drop=True, inplace=True)

display(len(df))
# display(df[:10])
print('done')

2090907

2090907

done


In [6]:
# cell-2 
# preparing data for finetuning


df['second_hemistich'].replace('', 'E', inplace=True)
dfc = df[['first_hemistich', 'second_hemistich', 'meter', 'link']].copy()
dfc['text'] = dfc['first_hemistich'] + ' S ' + dfc['second_hemistich']

#removing verses without a meter
dfc = dfc[dfc['meter'] != ''] 
dfc = dfc[dfc['meter'] != 'unspecified']
dfc = dfc[dfc['meter'] != 'mixed']



classic = ['taweel', 'kamel', 'baseet', 'khafif', 'wafer', 'rajaz', 'ramel', 'mutaqarib',
           'saree', 'munsarih', 'mujtath', 'hazaj', 'madeed', 'mutadarak', 'muqtadab', 'mudari'] 

#including only verses with classical meters
# dfc = dfc[dfc['meter'].isin(classic)]

dfc.reset_index(drop=True, inplace=True)

dfc['meter'] = dfc['meter'].astype('category')
# display(dfc['meter'].unique())

dfc['label'] = dfc['meter'].cat.codes #assign cat_value for each meter type
dftrain, dftest = train_test_split(dfc, test_size=0.20, random_state=42, stratify=dfc['label'])
ytrain = to_categorical(dftrain['label']).astype('int32')
ytest = to_categorical(dftest['label']).astype('int32')

max_sequence_length = 32
train_batch_size = 256
classes_num = len(dfc['meter'].unique())

display(len(dfc))
display(len(dftrain))
display(len(dftest))

1850351

1480280

370071

In [ ]:
#cell-3
#loading the tokenizer and the model

from transformers import AutoTokenizer,TFBertModel

tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabert')
bert = TFBertModel.from_pretrained('aubmindlab/bert-base-arabert', from_pt=True)


In [8]:
#cell-4
#tokenizing the data

xtrain = tokenizer(
    text=dftrain['text'].tolist(),
    add_special_tokens=True,
    max_length = max_sequence_length,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


xtest = tokenizer(
    text=dftest['text'].tolist(),
    add_special_tokens=True,
    max_length = max_sequence_length,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [7]:
# display(xtest)
display(xtest['input_ids'][0])

<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([    2,  2914,   277,   151,  2890,  1110,    83, 31553,  8365,
           9, 25033,   122,    85,  4147,  5444,   678,   321, 40125,
           3,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0], dtype=int32)>

In [9]:
#cell-5
# building classifier model

input_ids = Input(shape=(max_sequence_length,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_sequence_length,), dtype=tf.int32, name="attention_mask")

output = bert([input_ids, input_mask])[1] #pooled_output
output = tf.keras.layers.Dense(classes_num, activation='softmax', name='output')(output)
   
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=output)

optimizer = Adam(learning_rate=5e-05)


loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),

model.compile(optimizer = optimizer, loss = loss, metrics = metric)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 32)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  135193344   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 32,                                            

In [10]:
tf.config.experimental_run_functions_eagerly(True)
tf.config.run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [11]:
#train the model (classic meters)

train_history = model.fit(
    x ={'input_ids':xtrain['input_ids'],'attention_mask':xtrain['attention_mask']}, y = ytrain,
    validation_data = ({'input_ids':xtest['input_ids'],'attention_mask':xtest['attention_mask']}, 
    ytest), epochs=6, batch_size=train_batch_size )


Epoch 1/6
5783/5783 [==============================] - 1870s 323ms/step - loss: 0.3733 - balanced_accuracy: 0.8765 - val_loss: 0.1075 - val_balanced_accuracy: 0.9702
Epoch 2/6
5783/5783 [==============================] - 1875s 324ms/step - loss: 0.0974 - balanced_accuracy: 0.9726 - val_loss: 0.0830 - val_balanced_accuracy: 0.9781
Epoch 3/6
5783/5783 [==============================] - 1882s 325ms/step - loss: 0.0721 - balanced_accuracy: 0.9808 - val_loss: 0.0737 - val_balanced_accuracy: 0.9813
Epoch 4/6
5783/5783 [==============================] - 1854s 321ms/step - loss: 0.0598 - balanced_accuracy: 0.9848 - val_loss: 0.0749 - val_balanced_accuracy: 0.9814
Epoch 5/6
5783/5783 [==============================] - 1835s 317ms/step - loss: 0.0515 - balanced_accuracy: 0.9873 - val_loss: 0.0729 - val_balanced_accuracy: 0.9824
Epoch 6/6
5783/5783 [==============================] - 1839s 318ms/step - loss: 0.0459 - balanced_accuracy: 0.9890 - val_loss: 0.0726 - val_balanced_accuracy: 0.9825


In [12]:
#model evaluation (classic meters)

pred = model.predict({'input_ids':xtest['input_ids'],'attention_mask':xtest['attention_mask']})

y_pred = np.argmax(pred, axis = 1)
y_pred = to_categorical(y_pred, num_classes=classes_num).astype('int32')

print(classification_report(ytest, y_pred, digits=4))

11565/11565 [==============================] - 609s 53ms/step
              precision    recall  f1-score   support

           0     0.9913    0.9933    0.9923     53009
           1     0.8165    0.8838    0.8488      1833
           2     0.9839    0.9867    0.9853     81382
           3     0.9832    0.9920    0.9875     34821
           4     0.9098    0.8912    0.9004      1755
           5     0.5588    0.2639    0.3585        72
           6     0.9575    0.9551    0.9563      3871
           7     0.9753    0.9585    0.9668      5973
           8     0.8920    0.8263    0.8579       190
           9     0.9310    0.8650    0.8968      1326
          10     0.9832    0.9788    0.9810     13905
          11     0.9550    0.9262    0.9404     22267
          12     0.9704    0.9768    0.9736     17971
          13     0.9542    0.9670    0.9606     12252
          14     0.9943    0.9951    0.9947     88553
          15     0.9891    0.9849    0.9870     30891

   micro avg     0

In [9]:
#train the model (all meters)

train_history = model.fit(
    x ={'input_ids':xtrain['input_ids'],'attention_mask':xtrain['attention_mask']}, y = ytrain,
    validation_data = ({'input_ids':xtest['input_ids'],'attention_mask':xtest['attention_mask']}, 
    ytest), epochs=6, batch_size=train_batch_size )


Epoch 1/6
5975/5975 [==============================] - 1908s 319ms/step - loss: 0.4097 - balanced_accuracy: 0.8732 - val_loss: 0.1703 - val_balanced_accuracy: 0.9531
Epoch 2/6
5975/5975 [==============================] - 1897s 317ms/step - loss: 0.1516 - balanced_accuracy: 0.9577 - val_loss: 0.1320 - val_balanced_accuracy: 0.9643
Epoch 3/6
5975/5975 [==============================] - 1915s 320ms/step - loss: 0.1176 - balanced_accuracy: 0.9676 - val_loss: 0.1241 - val_balanced_accuracy: 0.9667
Epoch 4/6
5975/5975 [==============================] - 1932s 323ms/step - loss: 0.0980 - balanced_accuracy: 0.9734 - val_loss: 0.1227 - val_balanced_accuracy: 0.9678
Epoch 5/6
5975/5975 [==============================] - 1919s 321ms/step - loss: 0.0837 - balanced_accuracy: 0.9774 - val_loss: 0.1258 - val_balanced_accuracy: 0.9683
Epoch 6/6
5975/5975 [==============================] - 1906s 319ms/step - loss: 0.0722 - balanced_accuracy: 0.9807 - val_loss: 0.1243 - val_balanced_accuracy: 0.9687


In [10]:
#model evaluation (all meters)

pred = model.predict({'input_ids':xtest['input_ids'],'attention_mask':xtest['attention_mask']})

y_pred = np.argmax(pred, axis = 1)
y_pred = to_categorical(y_pred, num_classes=classes_num).astype('int32')

print(classification_report(ytest, y_pred, digits=4))

11950/11950 [==============================] - 646s 54ms/step
              precision    recall  f1-score   support

           0     0.9896    0.9906    0.9901     53009
           1     0.7554    0.5926    0.6641      1480
           2     0.8851    0.8927    0.8889       587
           3     0.7811    0.8381    0.8086      1686
           4     1.0000    0.1000    0.1818        10
           5     0.8175    0.8063    0.8119      1833
           6     0.9791    0.9868    0.9830     81382
           7     0.0000    0.0000    0.0000        12
           8     0.9764    0.9906    0.9834     34821
           9     0.0000    0.0000    0.0000         1
          10     0.9106    0.8650    0.8872      1755
          11     0.6325    0.4086    0.4965       257
          12     0.5846    0.5040    0.5413       377
          13     0.7350    0.6041    0.6632      7682
          14     0.3462    0.2500    0.2903        72
          15     0.9188    0.9359    0.9273      3871
          16     0.

In [ ]:

# classifier_path = 'finetuned/classic_meters_classifierTF_araBERT.h5'
classifier_path = 'finetuned/all_meters_classifierTF_araBERT.h5'


In [25]:
#saving finetuned model locally

model.save_weights(classifier_path)

In [26]:
#loading the finetuned model locally
# it's necessary to create new model similar to the saved one, then load it

from transformers import TFBertModel

def create_model():
    input_ids = Input(shape=(max_sequence_length,), dtype=tf.int32, name="input_ids")
    input_mask = Input(shape=(max_sequence_length,), dtype=tf.int32, name="attention_mask")
    bert = TFBertModel.from_pretrained('aubmindlab/bert-base-arabert', from_pt=True)
    
    output = bert([input_ids, input_mask])[1] #pooled_output
    output = tf.keras.layers.Dense(classes_num, activation='softmax', name='output')(output)
    
    model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=output)
    
    optimizer = Adam(learning_rate=5e-05)
     
    loss =CategoricalCrossentropy(from_logits = True)
    metric = CategoricalAccuracy('balanced_accuracy'),
    
    model.compile(optimizer = optimizer, loss = loss, metrics = metric)
    
    return model

model = create_model()
model.load_weights(classifier_path)

In [ ]:
#model evaluation

pred = model.predict({'input_ids':xtest['input_ids'],'attention_mask':xtest['attention_mask']})

y_pred = np.argmax(pred, axis = 1)
y_pred = to_categorical(y_pred, num_classes=classes_num).astype('int32')

print(classification_report(ytest, y_pred, digits=4))

In [ ]:

# further finetuning the model

train_history = model.fit(
    x ={'input_ids':xtrain['input_ids'],'attention_mask':xtrain['attention_mask']}, y = ytrain,
    validation_data = ({'input_ids':xtest['input_ids'],'attention_mask':xtest['attention_mask']}, 
    ytest), epochs=3, batch_size=train_batch_size )
